[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# Академия LangChain

Добро пожаловать в Академию LangChain!

## Контекст

В LangChain мы стремимся упростить создание LLM-приложений. Одним из типов LLM-приложений, которые вы можете создавать, являются агенты. **Интерес к созданию агентов** значительно вырос, поскольку они способны автоматизировать широкий спектр задач, которые ранее были недоступны.

Однако на практике создание систем, **устойчиво** справляющихся с этими задачами, оказывается крайне сложным. Работая с нашими пользователями над **внедрением агентов в промышленную эксплуатацию**, мы осознали, что часто требуется более **тонкий контроль**. Например, может потребоваться, чтобы агент всегда сначала вызывал определенный инструмент или использовал различные промпты в зависимости от своего состояния.

Для решения этой проблемы мы создали [LangGraph](https://docs.langchain.com/oss/python/langgraph/overview) — платформу для построения агентных и мультиагентных приложений. В отличие от пакета LangChain, основная философия LangGraph заключается в том, чтобы помочь разработчикам повысить **точность и управляемость** рабочих процессов агентов, что соответствует сложности реальных систем.

## Структура курса

Курс состоит из набора модулей, каждый из которых посвящен определенной теме, связанной с LangGraph. Для каждого модуля предусмотрена папка с серией блокнотов. К каждому блокноту прилагается видео с объяснением концепций, но при этом блокноты самодостаточны — они содержат все необходимые пояснения и могут изучаться независимо от видео. Каждая папка модуля также содержит папку "studio" с набором графов, которые можно загрузить в [LangSmith Studio](https://docs.langchain.com/langsmith/quick-start-studio) — нашу IDE-среду для разработки приложений LangGraph.

## Настройка

Перед началом работы выполните инструкции из файла README.md проекта, чтобы создать виртуальное окружение и установить зависимости.

## LLM-модели

В этом курсе мы будем использовать LLM-модели, которые принимают на вход последовательность сообщений и возвращают сообщения в качестве выходных данных. LangChain поддерживает множество моделей через [сторонние интеграции](https://docs.langchain.com/oss/python/integrations/chat). По умолчанию в курсе будет использоваться [ChatOpenAI](https://docs.langchain.com/oss/python/integrations/chat/openai), поскольку эта модель популярна и эффективна. Как уже упоминалось, убедитесь, что у вас есть `OPENAI_API_KEY`.

In [2]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_mistralai langchain_core langchain_community langchain-tavily

In [3]:
%pip list | wc -l

185
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# API ключи
KEY = "MISTRAL_API_KEY"
# KEY = "OPENAI_API_KEY"

_set_env(KEY)

MISTRAL_API_KEY:  ········


[Здесь](https://docs.langchain.com/oss/python/langchain/models) представлено полезное руководство по всем возможностям работы с чат-моделями, а ниже мы покажем несколько ключевых моментов. Если вы выполнили `pip install -r requirements.txt` как указано в README, значит, у вас установлен пакет "langchain-openai". С его помощью мы можем создать экземпляр нашей модели `ChatOpenAI`. С ценами на различные модели можно ознакомиться [здесь](https://openai.com/api/pricing/). По умолчанию в блокнотах используется "gpt-4o", поскольку эта модель предлагает оптимальное сочетание качества, стоимости и скорости, но вы также можете выбрать более бюджетную серию "gpt-3.5" или более новые модели.

Существует [несколько стандартных параметров](https://docs.langchain.com/oss/python/langchain/models#parameters), которые можно задать для чат-моделей. Два наиболее распространенных:

* `model`: название модели
* `temperature` (**температура**) — параметр сэмплирования

`temperature` управляет степенью случайности или креативности выходных данных модели. Низкое значение температуры (близкое к 0) обеспечивает более детерминированные и целенаправленные результаты, что хорошо подходит для задач, требующих точности и фактической достоверности. Высокая температура (близкая к 1) предпочтительна для творческих задач или генерации разнообразных ответов.

In [9]:
# from langchain_openai import ChatOpenAI
from langchain_mistralai import ChatMistralAI
mistral_small_latest = ChatMistralAI(model="mistral-small-latest", temperature=0.1)
mistral_medium_latest = ChatMistralAI(model="mistral-medium-latest", temperature=0.1)

У LLM-моделей в LangChain есть несколько [стандартных методов](https://reference.langchain.com/python/langchain_core/runnables). В основном мы будем использовать:

* [stream](https://docs.langchain.com/oss/python/langchain/models#stream) — **потоковая передача** ответа по мере генерации токенов
* [invoke](https://docs.langchain.com/oss/python/langchain/models#invoke) — **вызов** всей цепочки на входных данных

Как уже упоминалось, модели работают с [сообщениями](https://docs.langchain.com/oss/python/langchain/messages) в качестве входных данных. Сообщения содержат роль (которая описывает отправителя) и содержимое. Подробнее мы рассмотрим это позже, а сейчас давайте разберем основы.

In [6]:
from langchain_core.messages import HumanMessage

# Создать сообщение
msg = HumanMessage(content="Hello world", name="Lance")

# Список сообщений
messages = [msg]

# Вызов модели со списком сообщений
mistral_small_latest.invoke(messages)

AIMessage(content='Hello! 😊 How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 5, 'total_tokens': 18, 'completion_tokens': 13}, 'model_name': 'mistral-small-latest', 'model': 'mistral-small-latest', 'finish_reason': 'stop', 'model_provider': 'mistralai'}, id='lc_run--d34abbfe-9279-4eb2-9900-1e7d650b78d3-0', usage_metadata={'input_tokens': 5, 'output_tokens': 13, 'total_tokens': 18})

Мы получаем ответ `AIMessage`. Также обратите внимание, что мы можем просто вызвать модель, **передав ей строку**. Когда строка передается на вход, она автоматически преобразуется в `HumanMessage` и затем передается в базовую модель.


In [7]:
mistral_small_latest.invoke("hello world")

AIMessage(content='Hello! 😊 How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 5, 'total_tokens': 18, 'completion_tokens': 13}, 'model_name': 'mistral-small-latest', 'model': 'mistral-small-latest', 'finish_reason': 'stop', 'model_provider': 'mistralai'}, id='lc_run--1e4485b2-2620-4596-9969-6738446c21f1-0', usage_metadata={'input_tokens': 5, 'output_tokens': 13, 'total_tokens': 18})

In [10]:
mistral_medium_latest.invoke("hello world")

AIMessage(content='Hello, World! 🌍✨\n\nHow can I assist you today? 😊\n\n(If you\'re just starting with programming, here\'s a classic "Hello, World!" example in Python:)\n\n```python\nprint("Hello, World!")\n```', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 5, 'total_tokens': 61, 'completion_tokens': 56}, 'model_name': 'mistral-medium-latest', 'model': 'mistral-medium-latest', 'finish_reason': 'stop', 'model_provider': 'mistralai'}, id='lc_run--a2b25932-780f-492a-8a1e-f0476d85a5d4-0', usage_metadata={'input_tokens': 5, 'output_tokens': 56, 'total_tokens': 61})

Интерфейс одинаков для всех моделей, и модели обычно инициализируются один раз при запуске каждого файла (Jupyter Notebook).

Таким образом, вы можете легко переключаться между моделями, не изменяя исходный код, если предпочитаете другого поставщика.


## Инструменты поиска

Вы также увидите упоминание [Tavily](https://tavily.com/) в файле README. Это поисковая система, оптимизированная для работы с большими языковыми моделями (LLM) и RAG, целью которой является обеспечение эффективных, быстрых и устойчивых результатов поиска. Как уже упоминалось, зарегистрироваться очень просто, и сервис предлагает щедрый бесплатный тариф. Некоторые уроки (в Модуле 4) по умолчанию используют Tavily, но, конечно, вы можете использовать и другие поисковые инструменты, если захотите изменить код под свои нужды.

In [7]:
_set_env("TAVILY_API_KEY")

In [11]:
from langchain_tavily import TavilySearch  # updated at 1.0

tavily_search = TavilySearch(max_results=3)

data = tavily_search.invoke({"query": "What is LangGraph?"})
search_docs = data.get("results", data)

In [12]:
search_docs

[{'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
  'title': 'LangGraph Tutorial: What Is LangGraph and How to Use It?',
  'content': 'LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to focus on the high-level logic of their applications rather than the intricacies of agent coordination. Whether you need a chatbot that can handle various types of user requests or a multi-agent system that performs complex tasks, LangGraph provides the tools to build exactly what you need. LangGraph significantly simplifies the development of complex LLM applications by providing a structured framework for managing state and coordinating agent interactions.',
  'score': 0.9581988,
  'raw_content': None},
 {'url': 'https://www.geeksforgeeks.org/machine-learning